In [2]:
import glob
import os
import sys
sys.path.append("..")
import pandas as pd
from pandas.io.parsers import read_csv

from utils.configuration import Configuration

config = Configuration()
def add_file_name(csv, path):
    file_name = os.path.basename(path).split('.')[0]
    csv['config'] = file_name.split('result_')[1]
    return csv

In [3]:
def read_results():
    desc_map_files = []
    for path in glob.glob('../'+config.results_dir + "/evaluation/*.csv"):
        csv = read_csv(path, encoding='latin-1')
        csv = add_file_name(csv, path)
        desc_map_files.append(csv)
    results= pd.concat(desc_map_files, axis=0, ignore_index=True)
    results.drop('target_app', inplace=True, axis=1)
    results.drop('src_app', inplace=True, axis=1)
    results.fillna(0, inplace=True)
    return results

In [4]:
def config_average_performance(results):
    return results.groupby('config').mean().sort_values(by = ['accuracy'], ascending=False)

In [36]:
results = read_results()
agg_results = config_average_performance(results).sort_values(by = ['f1_score'], ascending=False)

In [37]:
config_stat = read_csv('../'+config.analyse_dir + '/configs_status.csv')
config_stat.set_index('config', inplace=True)
final_result = config_stat.join(agg_results)
final_result

,migrations,errors,blocking_errors,tp,tn,fp,fn,effort,accuracy,precision,recall,f1_score,reduction
config,,,,,,,,,,,,,
jaccard_empty_custom_atm,26,5,5,2.055556,2.833333,10.111111,2.000000,10.444444,0.394844,0.357070,0.529630,0.369089,-1.234259
wm_android_craftdroid_atm,27,7,7,2.055556,3.055556,10.000000,2.055556,10.444444,0.365199,0.258696,0.538889,0.331370,-1.305026
w2v_android_atm_atm,24,9,7,1.866667,2.666667,6.333333,2.333333,6.800000,0.388377,0.303196,0.483333,0.354889,-0.393016
wm_android_craftdroid_union,27,13,4,1.722222,3.111111,13.666667,2.333333,14.333333,0.324684,0.199055,0.464815,0.255549,-1.985185
w2v_googleplay_craftdroid_atm,27,12,10,2.153846,2.846154,10.384615,2.153846,11.000000,0.367305,0.304830,0.524359,0.350628,-1.506227
wm_googleplay_atm_union,24,7,5,2.062500,3.000000,12.625000,2.187500,13.250000,0.348628,0.275207,0.522917,0.317635,-1.884226
glove_googleplay_custom_atm,26,7,7,1.692308,2.615385,8.384615,2.461538,8.923077,0.343786,0.248400,0.424359,0.298034,-0.902198
wm_android_atm_atm,26,6,6,2.666667,3.222222,9.777778,1.666667,10.444444,0.410325,0.324375,0.637037,0.388277,-1.374206
glove_googleplay_atm_atm,27,6,6,1.722222,2.611111,7.222222,2.555556,8.500000,0.395441,0.344381,0.444444,0.355713,-0.511616


In [38]:
configs_mrr = read_csv('../' + config.analyse_dir + '/mrr.csv')
configs_mrr['config'] = configs_mrr[['word_embedding', 'training_set', 'algorithm','descriptors']].agg('_'.join, axis=1)
configs_mrr.drop(columns=['descriptors','word_embedding', 'training_set', 'algorithm'], inplace=True)
configs_mrr.set_index('config', inplace=True)
final_result = final_result.join(configs_mrr).sort_values(by = ['f1_score'], ascending=False)
# configs_mrr.loc[configs_mrr['config'] == 'glove_android_atm_union']
final_result.to_csv('../' + config.analyse_dir + '/final_results.csv')